# JUPYTERNOTEBOOK_INTERFACE_CONTROL_THE_ROBOT_IN_GAZEBO

In [1]:
%matplotlib widget
import numpy as npy
import collections
import jupyros as jr
import rospy
import time
import actionlib
import actionlib.msg
import rt2_assignment1.msg
import ipywidgets as widgets
import matplotlib.pyplot as plot
import matplotlib.ticker as ticker

from geometry_msgs.msg import Twist
from rt2_assignment1.srv import Command
from rosgraph_msgs.msg import Clock
from nav_msgs.msg import Odometry
from matplotlib import animation
from matplotlib import rc
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox ,Box ,AppLayout , GridspecLayout ,Label

#Control node initialization for the process
rospy.init_node('jupyros_node')


# Global variable initialization state
global linear_set, angular_set, rps

linear_set = 0.0 
angular_set = 0.0
reached = 0
cancelled = 0



# SLIDERS TO MODULATE THE LINEAR AND ANGULAR VELOCITY OF ROBOT

"go_to_point" ---> velocity modulation done with two servers(rando mode ) and in the manual mode
the topics were published with the cmd/vel topic of the robot

In [2]:
#sliders to controll the linear velocity and angular velocity

pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

msg = Twist()

Linear_vel = widgets.FloatSlider(min=0.0, max=1.0, description='L_v.')
display(Linear_vel)

Angular_vel = widgets.FloatSlider(min=0.0, max=1.0, description='A_v.')
display(Angular_vel)


def lin_c(change):
    if rps == False:
        msg.linear.x = change['new']
        pub.publish(msg)
    else:
        slider_linear = change['new']
        print("linear velocity altered")
    
def ang_c(change):
    if rps == False:
        msg.angular.z = change['new']
        pub.publish(msg)
    else:
        slider_angular = change['new']
        print(" angular velocity altered")

Linear_vel.observe(lin_c, names='value')
Angular_vel.observe(ang_c, names='value')

FloatSlider(value=0.0, description='L_v.', max=1.0)

FloatSlider(value=0.0, description='A_v.', max=1.0)

# CONTROL BUTTONS FOR ROBOT 

This block defines the implementation of robot controll using the widgets 


in which 

start --> turns the go_to_point for robot movement
stop -->  stops the robot


In [3]:
up  = Button(description='up',layout=Layout(width='auto', align="center", grid_area='8'),style=ButtonStyle(button_color='white'))
left  = Button(description='left',layout=Layout(width='auto', grid_area='4'),style=ButtonStyle(button_color='white'))
right = Button(description='right',layout=Layout(width='auto', grid_area='6'),style=ButtonStyle(button_color='white'))
down = Button(description='down',layout=Layout(width='auto', grid_area='2'),style=ButtonStyle(button_color='white'))
start  = Button(description='start',layout=Layout(width='auto', grid_area='start'),style=ButtonStyle(button_color='green'))
stop  = Button(description='Stop',layout=Layout(width='auto', grid_area='Stop'),style=ButtonStyle(button_color='red'))
output = widgets.Output()

#Node initialization
ui_client = rospy.ServiceProxy('/user_interface', Command)
# Stop the robot and initialize its status
ui_client("stop")
action_service = actionlib.SimpleActionServer('/go_to_point', rt2_assignment1.msg.RcAction, auto_start=False)
action_client = actionlib.SimpleActionClient('/go_to_point', rt2_assignment1.msg.RcAction)


# Output of the buttons such as control robot start and stop and all directional control
def start_(start):
    global rps,task_time
    with output:
        slider_lin = 1.0
        slider_ang = 1.0
    ui_client("start")
    rps=True
    print('robot started')
        

def stop_(stop):
    global rps
    with output:
        ui_client("stop")
        if rps == False:
            print("robot stopped")
        rps = False 
        print('robot stopped')
        
def up_(up):
    global rps, linear_set, angular_set
    if rps == True:
        ui_client("stop")
        print("input given start the robot to proceed the operation")
        rps = False
    elif rps == False:
        if linear_set == 0.0:
            print("Linear speed set to zero. Change it to make me move.")
        else:
            msg.linear.x = linear_set
            msg.angular.z = 0.0
            pub.publish(msg)
            print("robot_move straight")     

def down_(down):
    global rps, linear_set, angular_set
    if rps == True:
        ui_client("stop")
        print("input given start the robot to proceed the operation")
        rps = False
    elif rps == False:
        if linear_set == 0.0:
            print("Linear speed set to zero. Change it to make me move.")
        else:
            msg.linear.x = -linear_set
            msg.angular.z = angular_set
            linear_set = msg.linear.x 
            angular_set = msg.angular.z 
            pub.publish(msg)
            print("robot_move_backwards")
        
def right_(right):
    global rps, linear_set, angular_set
    if rps == True:
        ui_client("stop")
        print("input given start the robot to proceed the operation")
        rps = False
    elif rps == False:
        if angular_set == 0.0:
            print("Angular speed set to zero. Change it to make me turn.")
        else:
            msg.linear.x = linear_set
            msg.angular.z = angular_set
            pub.publish(msg)
            print("robot_turn_right")
        
def left_(left):
    global rps, linear_set, angular_set
    if rps == True:
        ui_client("stop")
        print("input given start the robot to proceed the operation")
        rps = False
    elif rps == False:
        if angular_set == 0.0:
            print("Angular speed set to zero. Change it to make me turn.")
        else:
            msg.linear.x = linear_set
            msg.angular.z = -angular_set
            pub.publish(msg)
            print("Robot_turn_left")

        
# Callback definition
start.on_click(start_)
stop.on_click(stop_)
up.on_click(up_)
down.on_click(down_)
right.on_click(right_)
left.on_click(left_)


buttons = AppLayout(header=start,left_sidebar=left,center=up,right_sidebar=right,footer=stop)
buttons1 = AppLayout(header=down)
widgets.VBox([buttons,buttons1, output])



# ROBOT CURRENT POSITION

this block tells the current position of the robot...... with the odometery topic.

In [4]:
 # Robot current position 
fig, ax = plot.subplots()
ax.set_xlim(( -20, 20))
ax.set_ylim((-20, 20))
ax.set_aspect("equal")
plot.xlabel("X-axis")
plot.ylabel("Y-axis")
plot.title("Robot current positon")

line, = ax.plot([], [], 'rx', markersize="5")

x_data =  collections.deque(maxlen=300)
y_data = collections.deque(maxlen=300)
theta_data = collections.deque(maxlen=300)

def init():
    line.set_data([], [])
    return (line,)
    
def animate(i):
    line.set_data(x_data, y_data)
    
    return (line)

def odom_callback(msg):
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x) 
      
                  
# Read robot odometry
jr.subscribe('/odom', Odometry, odom_callback)


anim_position = animation.FuncAnimation(fig, animate, init_func=init, blit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# DATA ANALYSIS OF LINEAR AND ANGULAR VELOCITY OF ROBOT

linear velocity and angular velocity data has been plotted .....

In [5]:
# Velocity feedback of the robot
fig_lin_vel, ax_lin_vel = plot.subplots()
ax_lin_vel.set_xlim(0, 100)
ax_lin_vel.set_ylim((-1.1, 1.1))
plot.title("Velocity check linear velocity")
plot.xlabel("Time [s]")
plot.ylabel("linear Velocity")

fig_ang_vel, ax_ang_vel = plot.subplots()
ax_ang_vel.set_xlim(0, 100)
ax_ang_vel.set_ylim((-1.1, 1.1))
plot.title("Velocity check angular velocity")
plot.xlabel("Time [s]")
plot.ylabel("angular Velocity")

# Array initilaization to store the value

time_vect = collections.deque(maxlen=400)
vel_target = collections.deque(maxlen=400)
vel_current = collections.deque(maxlen=400)
angular_target = collections.deque(maxlen=400)
angular_current = collections.deque(maxlen=400)

# linear velocity(v)
line_vel_target, = ax_lin_vel.plot([], [], '-b', markersize='5')
line_vel_current, = ax_lin_vel.plot([], [], '-r', markersize='5')
# angular velocity(w)
line_ang_target, = ax_ang_vel.plot([], [], 'xc', markersize='5')
line_ang_current, = ax_ang_vel.plot([], [], 'xg', markersize='5')


def init_vel():
   
    global line_vel_target, line_vel_current, line_ang_target, line_ang_current, ax_lin_vel, ax_ang_vel
    
    ax_lin_vel.legend()
    ax_lin_vel.set_title("Odom vs cmd_vel linear velocity")
    ax_lin_vel.set(xlabel='Time', ylabel='Linear velocity')
    ax_lin_vel.set_xlabel('time')

    ax_ang_vel.legend()
    ax_ang_vel.set_title("Odom vs cmd_vel angular velocity")
    ax_ang_vel.set(xlabel='Time', ylabel='Angular velocity')
    ax_ang_vel.set_xlabel('time')
    
    ax_lin_vel.set_ylim(-5,5)
    ax_ang_vel.set_ylim(-5,5)
     
    
    line_vel_target.set_data([],[])
    line_vel_current.set_data([],[])
    line_ang_current.set_data([],[])
    line_ang_target.set_data([],[])

    return (  line_vel_target,    line_vel_current, line_ang_current, line_ang_target)

def animate_vel(i):   
    ax_lin_vel.legend([line_vel_target ,line_vel_current], ['Cmd. linear', 'Linear Vel.'], loc = 'lower right')
    ax_ang_vel.legend([line_ang_target ,line_ang_current], ['Cmd. angular', 'ang Vel.'], loc = 'lower right')
    # linear velocity lines
    line_vel_target.set_data(time_vect, vel_target)   
    line_vel_current.set_data(time_vect, vel_current)  
    # angular velocity lines
    line_ang_target.set_data(time_vect, angular_target) 
    line_ang_current.set_data(time_vect, angular_current) 
    
    return (line_vel_target, line_ang_target, line_vel_current, line_ang_current)

def vel_callback(msg_cmd):   
    
    time_vect.append(rospy.get_time())       
    vel_target.append(msg_cmd.linear.x)  
    angular_target.append(msg_cmd.angular.z)  
    x_min = max([0, max(time_vect) - 20]) 
    x_max = max([100, max(time_vect)])
    ax_lin_vel.set_xlim( x_min, x_max )
    ax_ang_vel.set_xlim( x_min, x_max )

def odom_speed_callback(msg_odom):       
    vel_current.append(msg_odom.twist.twist.linear.x) 
    angular_current.append(-1.85*msg_odom.twist.twist.angular.z)

#robot_velocity data reading
jr.subscribe('cmd_vel', Twist, vel_callback)
#robot_odometery data reading
jr.subscribe('odom', Odometry, odom_speed_callback)

# Final output visual of odom and cmd_vel data
anim_l_velocity = animation.FuncAnimation(fig_lin_vel, animate_vel, frames = 1, init_func=init_vel, blit=True)

anim_a_velocity = animation.FuncAnimation(fig_ang_vel, animate_vel, frames = 1, init_func=init_vel, blit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# REACHED TARGETS AND CANCELLED TARGETS

In [6]:
# Callback and data for the barplot with the result reached and cancelled

hist_time = []
bar_y = [0, 0]

def result_callback(msg):
    if rps == True:
        bar_y[1] = bar_y[1] + 1 
        hist_time.append(msg.header.stamp.secs - msg.status.goal_id.stamp.secs)
    else:
        bar_y[0] = bar_y[0] + 1 

# Subscribe  the  topic go_to_point ansd its result

bar_fig = plot.figure()
ax_bar = bar_fig.add_subplot(111)
bar_x = ['Cancelled_Target', 'Reached_Target']
rects = plot.bar(bar_x, bar_y, color='c')

def animate_rects(i):
    for rec, val in zip(rects, bar_y):
         rec.set_height(val)   
    ax_bar.set_ylim([0, max(bar_y)+1]) 
    return rects, 

jr.subscribe('/go_to_point/result', rt2_assignment1.msg.RcAction, result_callback)

#final output visualization output
anim_reached = animation.FuncAnimation(bar_fig, animate_rects, blit=True)  

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# TIME TO REACH A GOAL BY ROBOT

In [7]:
# Time to reach the goal by robot ........
hist_fig, ax_hist = plot.subplots()
ax_hist.set_xlabel("Time for  target reaching")
ax_hist.set_ylabel("Occurencies")
bins=30

ax_hist.hist([],bins)


_, _, bar_container = ax_hist.hist(hist_time, bins, lw=1,ec="cyan", fc="green", alpha=0.5)


def animate_hist(i):    
        n, _ = npy.histogram(hist_time, bins)
        ax_hist.hist(hist_time, bins, lw=1,ec="cyan", fc="green", alpha=0.5)
        ax_hist.autoscale()
        

# output for the final visualization 
anim_hist = animation.FuncAnimation(hist_fig, animate_hist, blit=True) 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …